In [1]:
import folium
from folium import Map, Circle
import matplotlib.pyplot as plt
import pandas as pd
import json
import numpy as np
import warnings
warnings.filterwarnings('ignore')

ruta='/Users/nico/Desktop/Data Science/TFM/Energy Forecast/Datos Pablo/'
client=pd.read_csv(ruta+'client.csv')
elec=pd.read_csv(ruta+'electricity_prices.csv')
gas=pd.read_csv(ruta+'gas_prices.csv')
train=pd.read_csv(ruta+'train.csv')
prediccion=pd.read_csv(ruta+'forecast_weather.csv')
clima=pd.read_csv(ruta+'historical_weather.csv')
ubicaciones_n=pd.read_csv(ruta+'county_lon_lats.csv')
ubicaciones=pd.read_csv(ruta+'weather_station_to_county_mapping.csv')
trainn=train

In [3]:
mapa = folium.Map(location=[ubicaciones_n['latitude'].mean(), ubicaciones_n['longitude'].mean()], zoom_start=5)
county_unique=ubicaciones.county.unique()
# Actualizando la paleta de colores para asignar 'navy' a los valores NaN específicamente
colors = [f'#{np.random.randint(0, 0xFFFFFF):06x}' for x in county_unique if not np.isnan(x)]
colors.insert(0, 'navy')  # Asumiendo que el primer elemento en county_unique es NaN
county_unique_sorted = np.sort(county_unique[~np.isnan(county_unique)])
county_unique = np.insert(county_unique_sorted, 0, np.nan)  # Reinsertando NaN al principio si fue removido

# Mapeando cada 'county' a su color correspondiente, con 'navy' para NaN
county_to_color = dict(zip(county_unique, colors))

# Añadiendo puntos al mapa, con texto siempre visible para los valores no NaN de 'county' y color 'navy' para NaN.
for _, row in ubicaciones_n.iterrows():
    county = row['county']
    # Asignando 'navy' a NaNs o su color correspondiente a no NaNs
    color = 'navy' if np.isnan(county) else county_to_color[county]
    Circle(
        location=(row['latitude'], row['longitude']),
        radius=5000,
        color=color,
        fill=True,
        fill_color=color
    ).add_to(mapa)

    # Para valores no NaN, incluir el número de county siempre visible
    if not np.isnan(county):
        folium.map.Marker(
            [row['latitude'], row['longitude']],
            icon=folium.DivIcon(
                html=f"""<div style="font-size: 12pt; color : black">{int(county)}</div>"""
            )
        ).add_to(mapa)
mapa